In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import numpy as np

nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize

import json
import csv

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aligo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aligo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
allLabeledTweets = pd.read_csv('../jupyter/tweets/raw/allLabeledTweets.csv')
allLabeledTweets['label'] = allLabeledTweets['label'].astype(int)
allLabeledTweets['message'] = allLabeledTweets['message'].astype(str)
allLabeledTweets.head()

,message,id,tweetId,createdAt,language,inReplyToStatusId,inReplyToUserId,inReplyToScreenName,userId,userName,...,placeType,retweetedId,monitoringObjectId1,monitoringObjectId2,queryId,sentiment,retweetCount,label,clean_message,message_no_stopwords
0,Tiek vērtēti trīs potenciālie airBaltic invest...,1478404,925617390523732000,2017-11-01T08:55:57,lv,NaN,NaN,NaN,24855060,Dienas Bizness,...,NaN,NaN,389,389,NaN,0.0,0,0,tiek vērtēti trīs potenciālie airbaltic invest...,vērtēti trīs potenciālie airbaltic investori U...
1,"Vakardien, vēlu vakarā, ar svinīgu pasākumu ti...",1486476,924948828318511100,2017-10-30T12:39:20,lv,NaN,NaN,NaN,44849531,RIGA I Airport,...,NaN,NaN,703,703,NaN,-1.0,0,0,vakardien vēlu vakarā ar svinīgu pasākumu tika...,vakardien vēlu vakarā svinīgu pasākumu atklāts...
2,"Es ļoti ceru,ka potenciālie Air Baltic investo...",1488297,925794560420311000,2017-11-01T20:39:58,lv,NaN,NaN,NaN,218804015,Artis Pabriks,...,NaN,NaN,576,3,NaN,-1.0,9,2,es ļoti ceru ka potenciālie air baltic investo...,es ļoti ceru potenciālie air baltic investori ...
3,Ar kritisko domāšanu gan joprojām bēdīgi. Piln...,1489818,925045141018218500,2017-10-30T19:02:02,lv,NaN,NaN,NaN,21071644,Andris Rubīns,...,NaN,NaN,319,319,NaN,-1.0,3,2,ar kritisko domāšanu gan joprojām bēdīgi pilns...,kritisko domāšanu joprojām bēdīgi pilns facebo...
4,Latvijas lidsabiedrība airBaltic sadarbībā ar ...,1490250,925006964135952400,2017-10-30T16:30:20,lv,NaN,NaN,NaN,131562098,Latviesi.com,...,NaN,NaN,453,453,NaN,0.0,0,0,latvijas lidsabiedrība airbaltic sadarbībā ar ...,latvijas lidsabiedrība airbaltic sadarbībā apv...


In [5]:
allLabeledTweets['label'].value_counts()

0    253
2    136
1     87
Name: label, dtype: int64

In [14]:
allLabeledTweets.drop('monitoringObjectId1', inplace=True, axis=1)
allLabeledTweets.drop('monitoringObjectId2', inplace=True, axis=1)
allLabeledTweets.drop('queryId', inplace=True, axis=1)
allLabeledTweets.drop('sentiment', inplace=True, axis=1)
allLabeledTweets.drop('clean_message', inplace=True, axis=1)
allLabeledTweets.drop('message_no_stopwords', inplace=True, axis=1)

### Hashtags

In [7]:
def flatten(t):
    return [item for sublist in t for item in sublist]

popularHashtags = []

for index, row in allLabeledTweets.iterrows():
    m = row['message'].lower()
    popularHashtags.append(re.findall(r'\B(\#[a-zA-Z]+\b)(?!;)', m))
    
print(set(flatten(popularHashtags)))

{'#wtf', '#seb', '#dienasbizness', '#optibetvirsliga', '#swedbank', '#aset', '#veikalsdepo', '#bismartlv', '#ecb', '#bicg', '#manslmt', '#dodpieci', '#fintech', '#latvija', '#dus', '#nra', '#lmtstraume', '#rekonstrukcija', '#akropole', '#zekurbulis', '#continental', '#kaija', '#lmt', '#kartelis', '#gaismalatvijai', '#liepaja', '#huawei', '#dblv', '#futbols', '#socentlv', '#namejagredzens', '#oik', '#lauvassirds', '#latvenergo', '#diena', '#melot', '#lmtsmartfuture', '#akropolisgroup', '#olimpiadelv', '#teamlatvia', '#lieldraugs', '#lvpatriots', '#maxima', '#sportovisaklase', '#politika', '#optibet', '#mangali', '#lhf', '#vakance', '#unlockai', '#drogas', '#lielfans', '#valoda', '#rigvsjel', '#shocking', '#lietuvosenergija', '#tetaprojekts', '#labsserviss', '#circlek', '#airbaltic', '#db', '#zeltakivere', '#bite', '#z', '#dpd', '#teirdarbs', '#profesionalaiskretinisms', '#depo'}


### Lowercase

In [15]:
allLabeledTweets['message_lowercase'] = allLabeledTweets['message'].str.lower()
allLabeledTweets.head()

,message,id,tweetId,createdAt,language,inReplyToStatusId,inReplyToUserId,inReplyToScreenName,userId,userName,userScreenName,countryCode,placeName,placeFullName,placeType,retweetedId,retweetCount,label,message_lowercase
0,Tiek vērtēti trīs potenciālie airBaltic invest...,1478404,925617390523732000,2017-11-01T08:55:57,lv,NaN,NaN,NaN,24855060,Dienas Bizness,dblv,NaN,NaN,NaN,NaN,NaN,0,0,tiek vērtēti trīs potenciālie airbaltic invest...
1,"Vakardien, vēlu vakarā, ar svinīgu pasākumu ti...",1486476,924948828318511100,2017-10-30T12:39:20,lv,NaN,NaN,NaN,44849531,RIGA I Airport,riga_airport,NaN,NaN,NaN,NaN,NaN,0,0,"vakardien, vēlu vakarā, ar svinīgu pasākumu ti..."
2,"Es ļoti ceru,ka potenciālie Air Baltic investo...",1488297,925794560420311000,2017-11-01T20:39:58,lv,NaN,NaN,NaN,218804015,Artis Pabriks,Pabriks,NaN,NaN,NaN,NaN,NaN,9,2,"es ļoti ceru,ka potenciālie air baltic investo..."
3,Ar kritisko domāšanu gan joprojām bēdīgi. Piln...,1489818,925045141018218500,2017-10-30T19:02:02,lv,NaN,NaN,NaN,21071644,Andris Rubīns,Rubins,NaN,NaN,NaN,NaN,NaN,3,2,ar kritisko domāšanu gan joprojām bēdīgi. piln...
4,Latvijas lidsabiedrība airBaltic sadarbībā ar ...,1490250,925006964135952400,2017-10-30T16:30:20,lv,NaN,NaN,NaN,131562098,Latviesi.com,latviesicom,NaN,NaN,NaN,NaN,NaN,0,0,latvijas lidsabiedrība airbaltic sadarbībā ar ...


In [16]:
allLabeledTweets.to_csv('./tweets/allLabeledTweets.csv', index=False)

### Clean with regular expressions (new line symbol, URLs, numbers, hashtags, mentions, multiple whitespaces)

In [22]:
newLine ="\\n"
urls = '(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
numbers = '\d+((\.|\-)\d+)?'
mentions = '\B\@([\w\-]+)'
hashtag = '#'
whitespaces = '\s+'
leadTrailWhitespace = '^\s+|\s+?$'

allLabeledTweets['clean_message'] = allLabeledTweets['message_lowercase']
allLabeledTweets['clean_message'] = allLabeledTweets['clean_message'].str.replace(newLine,' ',regex=True)
allLabeledTweets['clean_message'] = allLabeledTweets['clean_message'].str.replace(urls,' URL ',regex=True)
allLabeledTweets['clean_message'] = allLabeledTweets['clean_message'].str.replace(mentions,' MENTION ',regex=True)
allLabeledTweets['clean_message'] = allLabeledTweets['clean_message'].str.replace(numbers,' NMBR ',regex=True)
allLabeledTweets['clean_message'] = allLabeledTweets['clean_message'].str.replace(hashtag,' ',regex=True)
allLabeledTweets['clean_message'] = allLabeledTweets['clean_message'].str.replace(whitespaces,' ',regex=True)
allLabeledTweets['clean_message'] = allLabeledTweets['clean_message'].str.replace(leadTrailWhitespace,'',regex=True)

In [25]:
print(allLabeledTweets.loc[345]['message'])
print(allLabeledTweets.loc[345]['clean_message'])
print(word_tokenize(allLabeledTweets.loc[345]['clean_message']))

Tiešraide jau pēc 15 minūtēm!

HK Prizma - HK Zemgale/LU 
Optibet hokeja līga

Skaties spēli šeit - https://t.co/RvtA1bPWF7 https://t.co/21nbpZtHd9
tiešraide jau pēc NMBR minūtēm! hk prizma - hk zemgale/lu optibet hokeja līga skaties spēli šeit - URL URL
['tiešraide', 'jau', 'pēc', 'NMBR', 'minūtēm', '!', 'hk', 'prizma', '-', 'hk', 'zemgale/lu', 'optibet', 'hokeja', 'līga', 'skaties', 'spēli', 'šeit', '-', 'URL', 'URL']


In [24]:
allLabeledTweets.to_csv('./tweets/allLabeledTweets.csv', index=False)

### No punctuation

In [26]:
punctuationAfterWord = '(?<=\s)|(?<=^)|(?<=\b)(?:[-\'.%$#&,\/:!-?`“–&]\b|\b[-\'.%$#&,\/:!-?`“–&])+(?=\s|$|\b)'
punctuation = '[(,|.|!|\-|\–|\`|\\|\/|?|:|“|”|►|;|\'|%|&|@)]'
smile = ':\)|=\)'
frown = ':\(|=\('

allLabeledTweets['clean_message_no_punct'] = allLabeledTweets['clean_message']
allLabeledTweets['clean_message_no_punct'] = allLabeledTweets['clean_message_no_punct'].str.replace(smile,'SMILE',regex=True)
allLabeledTweets['clean_message_no_punct'] = allLabeledTweets['clean_message_no_punct'].str.replace(frown,'FROWN',regex=True)
allLabeledTweets['clean_message_no_punct'] = allLabeledTweets['clean_message_no_punct'].str.replace(punctuation,' ',regex=True)
allLabeledTweets['clean_message_no_punct'] = allLabeledTweets['clean_message_no_punct'].str.replace(punctuationAfterWord,'',regex=True)

In [27]:
print(allLabeledTweets.loc[345]['clean_message_no_punct'])
print(word_tokenize(allLabeledTweets.loc[345]['clean_message_no_punct']))

tiešraide jau pēc NMBR minūtēm  hk prizma   hk zemgale lu optibet hokeja līga skaties spēli šeit   URL URL
['tiešraide', 'jau', 'pēc', 'NMBR', 'minūtēm', 'hk', 'prizma', 'hk', 'zemgale', 'lu', 'optibet', 'hokeja', 'līga', 'skaties', 'spēli', 'šeit', 'URL', 'URL']


In [28]:
allLabeledTweets.to_csv('./tweets/allLabeledTweets.csv', index=False)

### Stopwords

#### From list

##### With punctuation

In [30]:
stopwords_lv = ["ārpus", "šaipus", "aiz", "ap", "apakš", "apakšpus", "arī", "ar", "ar", "augšpus", "būšu", "būs", "būsi", "būsiet", "būsim", "būt", "bet", "bez", "bijām", "bijāt", "bija", "biji", "biju", "caur", "dēļ", "diemžēl", "diezin", "droši", "esam", "esat", "esi", "esmu", "gan", "gar", "iekām", "iekāms", "iekš", "iekšpus", "iekam", "iekams", "ik", "ir", "it", "itin", "iz", "jā", "ja", "jau", "jebšu", "jeb", "jel", "jo", "kā", "kļūšu", "kļūs", "kļūsi", "kļūsiet", "kļūsim", "kļūst", "kļūstam	kļūstat", "kļūsti", "kļūstu", "kļūt", "kļuvām", "kļuvāt", "kļuva", "kļuvi", "kļuvu", "ka", "kamēr", "kaut", "kolīdz", "kopš", "līdz", "līdzko", "labad", "lai", "lejpus", "nē", "ne", "nebūt", "nedz", "nekā", "nevis", "nezin", "no", "nu", "otrpus", "pār", "pēc", "pa", "par", "pat", "pie", "pirms", "pret", "priekš", "starp", "tā", "tādēļ", "tālab", "tāpēc", "taču", "tad", "tak", "tapāt", "tapšu", "tapi", "taps", "tapsi", "tapsiet", "tapsim", "tapt", "te", "tiec", "tiek", "tiekam", "tiekat", "tieku	tikām", "tikāt", "tikšu", "tik", "tika", "tikai", "tiki", "tikko", "tiklīdz", "tiklab", "tiks", "tiksiet", "tiksim", "tikt", "tiku", "tikvien", "tomēr", "topat", "turpretī", "turpretim", "un", "uz", "vai", "varēšu", "varējām", "varējāt", "varēja", "varēji", "varēju", "varēs", "varēsi", "varēsiet", "varēsim", "varēt", "var", "varat", "viņpus", "vien", "vien", "virs", "virspus", "vis", "zem"]
message_no_stopwords=[]

for message in allLabeledTweets.clean_message:
    text_tokens = word_tokenize(message)
    tokens_without_stop_words = [word for word in text_tokens if not word in stopwords_lv]
    message_no_stopwords.append((" ").join(tokens_without_stop_words))

allLabeledTweets['clean_message_no_stopwords_from_list'] = message_no_stopwords

In [31]:
print(allLabeledTweets.loc[345]['clean_message_no_stopwords_from_list'])
print(word_tokenize(allLabeledTweets.loc[345]['clean_message_no_stopwords_from_list']))

tiešraide NMBR minūtēm ! hk prizma - hk zemgale/lu optibet hokeja līga skaties spēli šeit - URL URL
['tiešraide', 'NMBR', 'minūtēm', '!', 'hk', 'prizma', '-', 'hk', 'zemgale/lu', 'optibet', 'hokeja', 'līga', 'skaties', 'spēli', 'šeit', '-', 'URL', 'URL']


In [32]:
allLabeledTweets.to_csv('./tweets/allLabeledTweets.csv', index=False)

##### No punctuation

In [34]:
stopwords_lv = ["ārpus", "šaipus", "aiz", "ap", "apakš", "apakšpus", "arī", "ar", "ar", "augšpus", "būšu", "būs", "būsi", "būsiet", "būsim", "būt", "bet", "bez", "bijām", "bijāt", "bija", "biji", "biju", "caur", "dēļ", "diemžēl", "diezin", "droši", "esam", "esat", "esi", "esmu", "gan", "gar", "iekām", "iekāms", "iekš", "iekšpus", "iekam", "iekams", "ik", "ir", "it", "itin", "iz", "jā", "ja", "jau", "jebšu", "jeb", "jel", "jo", "kā", "kļūšu", "kļūs", "kļūsi", "kļūsiet", "kļūsim", "kļūst", "kļūstam	kļūstat", "kļūsti", "kļūstu", "kļūt", "kļuvām", "kļuvāt", "kļuva", "kļuvi", "kļuvu", "ka", "kamēr", "kaut", "kolīdz", "kopš", "līdz", "līdzko", "labad", "lai", "lejpus", "nē", "ne", "nebūt", "nedz", "nekā", "nevis", "nezin", "no", "nu", "otrpus", "pār", "pēc", "pa", "par", "pat", "pie", "pirms", "pret", "priekš", "starp", "tā", "tādēļ", "tālab", "tāpēc", "taču", "tad", "tak", "tapāt", "tapšu", "tapi", "taps", "tapsi", "tapsiet", "tapsim", "tapt", "te", "tiec", "tiek", "tiekam", "tiekat", "tieku	tikām", "tikāt", "tikšu", "tik", "tika", "tikai", "tiki", "tikko", "tiklīdz", "tiklab", "tiks", "tiksiet", "tiksim", "tikt", "tiku", "tikvien", "tomēr", "topat", "turpretī", "turpretim", "un", "uz", "vai", "varēšu", "varējām", "varējāt", "varēja", "varēji", "varēju", "varēs", "varēsi", "varēsiet", "varēsim", "varēt", "var", "varat", "viņpus", "vien", "vien", "virs", "virspus", "vis", "zem"]
message_no_stopwords=[]

for message in allLabeledTweets.clean_message_no_punct:
    text_tokens = word_tokenize(message)
    tokens_without_stop_words = [word for word in text_tokens if not word in stopwords_lv]
    message_no_stopwords.append((" ").join(tokens_without_stop_words))

allLabeledTweets['clean_message_no_punct_no_stopwords_from_list'] = message_no_stopwords

In [35]:
print(allLabeledTweets.loc[345]['clean_message_no_punct_no_stopwords_from_list'])
print(word_tokenize(allLabeledTweets.loc[345]['clean_message_no_punct_no_stopwords_from_list']))

tiešraide NMBR minūtēm hk prizma hk zemgale lu optibet hokeja līga skaties spēli šeit URL URL
['tiešraide', 'NMBR', 'minūtēm', 'hk', 'prizma', 'hk', 'zemgale', 'lu', 'optibet', 'hokeja', 'līga', 'skaties', 'spēli', 'šeit', 'URL', 'URL']


In [36]:
allLabeledTweets.to_csv('./tweets/allLabeledTweets.csv', index=False)

#### From most frequently used words excluding brand names

In [43]:
brands = [
  "latvenergo",
  "rimi",
  "maxima",
  "swedbank",
  "lmt",
  "tet",
  "lg",
  "tele2",
  "airbaltic",
  "olympic",
  "seb",
  "grindeks",
  "citadele",
  "bite",
  "drogas",
  "depo",
  "circlek",
  "lb",
  "optibet",
  "evolution",
];

In [62]:
def getWordFrequency(processedTweetOutputFile):
    tweets = pd.read_csv(processedTweetOutputFile)

    allPostsConcat = ''
    for tweet in tweets.clean_message_no_punct:
        if(type(tweet)==str):
            allPostsConcat+=' '+ tweet

    # create bag-of-words
    all_words = []

    words = word_tokenize(allPostsConcat)
    words = [word for word in words if not word in brands]
    for word in words:
        if word!='``' and word!="''" and len(word) > 1:
            all_words.append(word)

    all_words = nltk.FreqDist(all_words)
    
    return all_words

In [63]:
freqWords = getWordFrequency('./tweets/allLabeledTweets.csv')

# print the total number of words and the 100 most common words
print('Number of words: {}'.format(len(freqWords)))
print('Most common words: {}'.format(freqWords.most_common(100)))

Number of words: 3834
Most common words: [('URL', 510), ('MENTION', 329), ('NMBR', 324), ('un', 191), ('ir', 118), ('ar', 117), ('par', 111), ('no', 80), ('ka', 73), ('uz', 50), ('jau', 49), ('vai', 48), ('arī', 43), ('kā', 42), ('kas', 41), ('vēl', 40), ('kopā', 40), ('latvijas', 39), ('pēc', 38), ('bet', 38), ('nav', 34), ('lieliskamūzika', 34), ('skaties', 31), ('to', 28), ('tā', 26), ('tiešraide', 26), ('lhf', 25), ('pie', 24), ('kopāspēks', 24), ('spēli', 24), ('ja', 23), ('es', 22), ('tas', 22), ('tad', 22), ('šeit', 22), ('būs', 20), ('the', 20), ('lielfans', 19), ('minūtēm', 19), ('lai', 18), ('tagad', 18), ('man', 17), ('šodien', 17), ('tikai', 16), ('hokeja', 16), ('ļoti', 15), ('rīgas', 15), ('mūsu', 15), ('oik', 15), ('līga', 15), ('kad', 14), ('eiro', 14), ('gada', 13), ('var', 13), ('ne', 12), ('izlase', 12), ('nekā', 12), ('gan', 11), ('pret', 11), ('vairāk', 11), ('dienasbizness', 11), ('ielā', 11), ('tv', 11), ('kur', 10), ('darbinieku', 10), ('latvija', 10), ('amp', 1

In [66]:
stopwordsFromFreqList = ['un','ir','ar','par','no', 'ka', 'uz','jau','vai','arī','kā','kas','vēl', 'kopā','pēc', 'bet', 'to','tā','pie','ja','es', 'tas','tad','šeit', 'būs', 'the','lai','tagad','man','šodien','tikai', 'mūsu','kad','nekā','gan','kur','ko','bija','līdz','nu','jo','kāpēc','kāds','pa']
len(stopwordsFromFreqList)

44

##### With punctuation

In [71]:
message_no_stopwords=[]

for message in allLabeledTweets.clean_message:
    text_tokens = word_tokenize(message)
    tokens_without_stop_words = [word for word in text_tokens if not word in stopwordsFromFreqList]
    message_no_stopwords.append((" ").join(tokens_without_stop_words))

allLabeledTweets['clean_message_no_freq_stopwords'] = message_no_stopwords

In [72]:
print(allLabeledTweets.loc[345]['clean_message_no_freq_stopwords'])
print(word_tokenize(allLabeledTweets.loc[345]['clean_message_no_freq_stopwords']))

tiešraide NMBR minūtēm ! hk prizma - hk zemgale/lu optibet hokeja līga skaties spēli - URL URL
['tiešraide', 'NMBR', 'minūtēm', '!', 'hk', 'prizma', '-', 'hk', 'zemgale/lu', 'optibet', 'hokeja', 'līga', 'skaties', 'spēli', '-', 'URL', 'URL']


In [73]:
allLabeledTweets.to_csv('./tweets/allLabeledTweets.csv', index=False)

##### No punctuation

In [67]:
message_no_stopwords=[]

for message in allLabeledTweets.clean_message_no_punct:
    text_tokens = word_tokenize(message)
    tokens_without_stop_words = [word for word in text_tokens if not word in stopwordsFromFreqList]
    message_no_stopwords.append((" ").join(tokens_without_stop_words))

allLabeledTweets['clean_message_no_punct_no_freq_stopwords'] = message_no_stopwords

In [68]:
print(allLabeledTweets.loc[345]['clean_message_no_punct_no_freq_stopwords'])
print(word_tokenize(allLabeledTweets.loc[345]['clean_message_no_punct_no_freq_stopwords']))

tiešraide NMBR minūtēm hk prizma hk zemgale lu optibet hokeja līga skaties spēli URL URL
['tiešraide', 'NMBR', 'minūtēm', 'hk', 'prizma', 'hk', 'zemgale', 'lu', 'optibet', 'hokeja', 'līga', 'skaties', 'spēli', 'URL', 'URL']


In [69]:
allLabeledTweets.to_csv('./tweets/allLabeledTweets.csv', index=False)